In [1]:
!pip3 install --upgrade langchain langchain_experimental openai --quiet 
!pip3 install --upgrade matplotlib --quiet
!pip3 install --upgrade pandas --quiet
!pip3 install --upgrade sqlalchemy --quiet


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip3 install --upgrade pip


In [2]:
import os
import csv
import pandas as pd
from datetime import datetime
from langchain_openai import OpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
import sqlalchemy

In [3]:
# initialise sql engine
conn = sqlalchemy.create_engine('sqlite:////Users/davidrinaldi/Documents/Projects/myk-dashboard/dev/data/full_iseller_data.db')

In [4]:
def WeekOfMonth(date):
    first_day_of_month = date.replace(day=1)
    day_of_week_first_day = first_day_of_month.weekday()
    week_number = (date.day + day_of_week_first_day - 1) // 7 + 1
    return week_number

In [5]:
# import csv and convert to db
iseller_data = pd.read_excel('/Users/davidrinaldi/Documents/Projects/myk-dashboard/data/full_iseller_data.xlsx')
# add order date data
iseller_data['closed_date'] = pd.to_datetime(iseller_data['closed_date'])# + pd.Timedelta(hours=7)
iseller_data['hour'] = iseller_data['closed_date'].dt.hour
iseller_data['day'] = iseller_data['closed_date'].dt.day
iseller_data['month'] = iseller_data['closed_date'].dt.month
iseller_data['week'] = iseller_data['closed_date'].dt.isocalendar().week
iseller_data['year'] = iseller_data['closed_date'].dt.year
iseller_data['quarter'] = iseller_data['closed_date'].dt.quarter
iseller_data['order_date'] = pd.to_datetime(iseller_data['closed_date'])
iseller_data['week_of_month'] = iseller_data['closed_date'].apply(WeekOfMonth)
iseller_data['month_name'] = iseller_data['closed_date'].dt.strftime('%B')
iseller_data['quarter_year'] = iseller_data.apply(lambda row: f"Q{row['quarter']}-{row['year']}", axis=1)

In [6]:
#db = SQLDatabase.from_uri("sqlite:///data/full_iseller_data.db")


In [7]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI, OpenAI

query_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.9)
analysis_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

## location table

In [8]:
# initialise location tables
from scripts.utils import generate_location_df

conn = sqlalchemy.create_engine('sqlite:////Users/davidrinaldi/Documents/Projects/myk-dashboard/dev/data/temp.db')
location_df = generate_location_df(iseller_data)
location_df.to_sql('locations', conn, if_exists='replace', index=False)
db = SQLDatabase.from_uri("sqlite:///data/temp.db")


In [9]:
location_agent = create_sql_agent(query_llm, db=db, agent_type="openai-tools", verbose=True)

In [10]:
response = location_agent.invoke(
    """
    Generate only three additional prompts that could be asked to understand or analysis the data.
        Output the three prompts in list format counting up from 1 (1. prompt\n 2. prompt\n 3. prompt\n). 
        Generate only the prompts in the list, exclude their index.
        End the response with <EOF>.
        Do not respond with an empty result or say "I don't know".
    """
)
print(response['output'])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


locations
Invoking: `sql_db_schema` with `{'table_names': 'locations'}`



CREATE TABLE locations (
	outlet_name TEXT, 
	year INTEGER, 
	month_name TEXT, 
	month INTEGER, 
	quantity BIGINT, 
	quantity_diff FLOAT, 
	growth_rate FLOAT
)

/*
3 rows from locations table:
outlet_name	year	month_name	month	quantity	quantity_diff	growth_rate
Ayam! Alun-Alun BKS	2024	March	3	172	None	None
Ayam! Alun-Alun BKS	2024	April	4	134	-38.0	-22.093023255813954
Ayam! Alun-Alun BKS	2024	May	5	23	-111.0	-82.83582089552239
*/1. Which outlet had the highest quantity of products sold in a single month?
2. How does the growth rate differ between outlets in different years?
3. Is there a specific month where most outlets experienced a decrease in quantity compared to the previous month?
<EOF>

> Finished chain.
1. Which outlet had the highest quantity of products sold in a single month?
2. How does the growth rate differ be

In [11]:
location_questions = response['output'].split('<EOF>')[0].split('\n')[:3]
location_questions

['1. Which outlet had the highest quantity of products sold in a single month?',
 '2. How does the growth rate differ between outlets in different years?',
 '3. Is there a specific month where most outlets experienced a decrease in quantity compared to the previous month?']

In [12]:
# create location answer model
location_query = create_sql_agent(analysis_llm, db=db, agent_type="tool-calling")

In [13]:
location_responses = []
for question in location_questions:
    print(question)
    response = location_query.invoke(question + "All prices are in IDR")
    location_responses.append(response['output'])


1. Which outlet had the highest quantity of products sold in a single month?
2. How does the growth rate differ between outlets in different years?
3. Is there a specific month where most outlets experienced a decrease in quantity compared to the previous month?


In [14]:
print(location_responses[0])
    

The outlet "Bekasi - Perjuangan" had the highest quantity of products sold in a single month, with a quantity of 11,672 products.


In [15]:
print(location_responses[1])


The growth rates for outlets in different years are as follows:

1. Outlet: Ayam! Alun-Alun BKS, Year: 2024, Growth Rate: None
2. Outlet: Ayam! Alun-Alun BKS, Year: 2024, Growth Rate: -22.09%
3. Outlet: Ayam! Alun-Alun BKS, Year: 2024, Growth Rate: -82.84%
4. Outlet: Ayam! Dapur Kitchen BKS, Year: 2024, Growth Rate: None
5. Outlet: Ayam! Dapur Kitchen BKS, Year: 2024, Growth Rate: -9.56%
6. Outlet: Ayam! Dapur Kitchen BKS, Year: 2024, Growth Rate: 58.31%
7. Outlet: Ayam! Dapur Kitchen BKS, Year: 2024, Growth Rate: 26.34%
8. Outlet: Ayam! Dapur Kitchen BKS, Year: 2024, Growth Rate: -93.05%
9. Outlet: Ayam! Libersa Park BKS, Year: 2024, Growth Rate: None
10. Outlet: Ayam! Libersa Park BKS, Year: 2024, Growth Rate: 14.05%


In [16]:
print(location_responses[2])


The month where most outlets experienced a decrease in quantity compared to the previous month is May, with 9 outlets showing a decrease.


In [17]:
# Function to escape newlines
def escape_newlines(text):
    return text.replace('\n', '\\n')

with open('outputs/location.csv', 'w', newline='', encoding='utf-8') as fopen:
    writer = csv.writer(fopen, quoting=csv.QUOTE_MINIMAL)
    
    # Write header if the file is empty
    fopen.seek(0, os.SEEK_END)
    if fopen.tell() == 0:
        writer.writerow(['id', 'prompt', 'response'])

    # Write each question and response to the CSV file
    for idx in range(len(location_responses)):
        writer.writerow(
            [
                idx + 1, 
                escape_newlines(location_questions[idx]), 
                escape_newlines(location_responses[idx])
            ]
        )

## sales table

In [18]:
# initialise sales tables
from scripts.utils import generate_sales_table

conn = sqlalchemy.create_engine('sqlite:////Users/davidrinaldi/Documents/Projects/myk-dashboard/dev/data/temp2.db')
sales_df = generate_sales_table(iseller_data)
sales_df.to_sql('sales', conn, if_exists='replace', index=False)
db = SQLDatabase.from_uri("sqlite:///data/temp2.db")

In [19]:
sales_agent = create_sql_agent(query_llm, db=db, agent_type="openai-tools", verbose=True)

In [20]:
response = sales_agent.invoke(
    """
    Generate only three additional prompts that could be asked to understand or analysis the data.
        Output the three prompts in list format counting up from 1 (1. prompt\n 2. prompt\n 3. prompt\n). 
        Generate only the prompts in the list, exclude their index.
        End the response with <EOF>.
        Do not respond with an empty result or say "I don't know".
    """
)
print(response['output'])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


sales
Invoking: `sql_db_schema` with `{'table_names': 'sales'}`



CREATE TABLE sales (
	clean_product_name TEXT, 
	product_type TEXT, 
	total_revenue BIGINT, 
	current_quarter_revenue FLOAT, 
	last_quarter_revenue FLOAT, 
	current_month_revenue FLOAT, 
	last_month_revenue FLOAT, 
	quarterly_growth FLOAT, 
	monthly_growth FLOAT, 
	"quarter_Growth(%)" FLOAT, 
	"Monthly_Growth(%)" FLOAT
)

/*
3 rows from sales table:
clean_product_name	product_type	total_revenue	current_quarter_revenue	last_quarter_revenue	current_month_revenue	last_month_revenue	quarterly_growth	monthly_growth	quarter_Growth(%)	Monthly_Growth(%)
(es batu) crystalline cup	REGULAR	201033098	55569433.0	145463665.0	10521000.0	45048433.0	-61.79840993281724	-76.64513658000047	-0.45	-0.62
a&w root beer kaleng	REGULAR	21235100	5086300.0	16148800.0	89000.0	4997300.0	-68.50354205885267	-98.21903828067157	-0.52	-0.97
absolut vodka	VODKA	515999

In [21]:
sales_questions = response['output'].split('<EOF>')[0].split('\n')[:3]
sales_questions

['1. What are the top three product types by total revenue in the current quarter?',
 '2. How much percentage growth has the product "a&w root beer kaleng" seen in the last month compared to the previous month?',
 '3. Can you provide the products that have experienced negative quarterly growth?  ']

In [22]:
# create location answer model
sales_query = create_sql_agent(analysis_llm, db=db, agent_type="tool-calling")

In [23]:
sales_responses = []
for question in sales_questions:
    print(question)
    response = sales_query.invoke(question + "All prices are in IDR")
    sales_responses.append(response['output'])

1. What are the top three product types by total revenue in the current quarter?
2. How much percentage growth has the product "a&w root beer kaleng" seen in the last month compared to the previous month?
3. Can you provide the products that have experienced negative quarterly growth?  


In [24]:
print(sales_responses[0])

The top three product types by total revenue in the current quarter are:

1. WHISKY with total revenue of 2,721,392,800 IDR
2. BEER with total revenue of 2,623,261,150 IDR
3. ANGGUR with total revenue of 2,444,051,700 IDR


In [25]:
print(sales_responses[1])

The product "a&w root beer kaleng" has seen a monthly growth rate of -97% in the last month compared to the previous month.


In [26]:
print(sales_responses[2])

The products that have experienced negative quarterly growth are as follows:

1. Product: (es batu) crystalline cup, Product Type: REGULAR, Quarterly Growth: -61.80%
2. Product: a&w root beer kaleng, Product Type: REGULAR, Quarterly Growth: -68.50%
3. Product: absolut vodka, Product Type: VODKA, Quarterly Growth: -45.63%
4. Product: alexis anggur hijau, Product Type: ANGGUR, Quarterly Growth: -71.64%
5. Product: alexis anggur merah, Product Type: ANGGUR, Quarterly Growth: -65.27%
6. Product: alva vodka original, Product Type: VODKA, Quarterly Growth: -51.54%
7. Product: andong strength soju green tea, Product Type: SOJU, Quarterly Growth: -46.32%
8. Product: andong strength soju lychee, Product Type: SOJU, Quarterly Growth: -41.84%
9. Product: anggur ginseng intisari, Product Type: ANGGUR, Quarterly Growth: -68.86%
10. Product: anggur ginseng intisari, Product Type: WINE, Quarterly Growth: -27.87%


In [27]:
# Function to escape newlines
def escape_newlines(text):
    return text.replace('\n', '\\n')

with open('outputs/sales.csv', 'w', newline='', encoding='utf-8') as fopen:
    writer = csv.writer(fopen, quoting=csv.QUOTE_MINIMAL)
    
    # Write header if the file is empty
    fopen.seek(0, os.SEEK_END)
    if fopen.tell() == 0:
        writer.writerow(['id', 'prompt', 'response'])

    # Write each question and response to the CSV file
    for idx in range(len(location_responses)):
        writer.writerow(
            [
                idx + 1, 
                escape_newlines(sales_questions[idx]), 
                escape_newlines(sales_responses[idx])
            ]
        )

## Top Product by Quantity Sold

In [28]:
# Top Product by Quantity Sold
tpqs = iseller_data.groupby(['clean_product_name', 'year', 'month', 'week']).agg({'total_amount': 'mean', 'quantity': 'sum'}).reset_index()\
        .sort_values(by='quantity', ascending=False).sort_values(by=["year", "month", "week"])
tpqs.head()

,clean_product_name,year,month,week,total_amount,quantity
3595,kawa kawa anggur hijau,2024,1,1,140173.195980,557
340,anggur merah gold,2024,1,1,131484.556314,395
457,api anggur hijau,2024,1,1,164322.362245,292
296,anggur merah,2024,1,1,118656.692308,231
1125,bintang kaleng,2024,1,1,222735.696429,224


In [29]:
conn = sqlalchemy.create_engine('sqlite:////Users/davidrinaldi/Documents/Projects/myk-dashboard/dev/data/temp3.db')
tpqs.to_sql('top_product_by_quantity_sold', conn, if_exists='replace', index=False)
db = SQLDatabase.from_uri("sqlite:///data/temp3.db")

In [30]:
tpqs_agent = create_sql_agent(query_llm, db=db, agent_type="openai-tools", verbose=True)

In [31]:
response = tpqs_agent.invoke(
    """
    Generate only three additional prompts that could be asked to understand or analysis the data.
        Output the three prompts in list format counting up from 1 (1. prompt\n 2. prompt\n 3. prompt\n). 
        Generate only the prompts in the list, exclude their index.
        End the response with <EOF>.
        Do not respond with an empty result or say "I don't know".
    """
)
print(response['output'])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


top_product_by_quantity_sold
Invoking: `sql_db_schema` with `{'table_names': 'top_product_by_quantity_sold'}`



CREATE TABLE top_product_by_quantity_sold (
	clean_product_name TEXT, 
	year INTEGER, 
	month INTEGER, 
	week BIGINT, 
	total_amount FLOAT, 
	quantity BIGINT
)

/*
3 rows from top_product_by_quantity_sold table:
clean_product_name	year	month	week	total_amount	quantity
kawa kawa anggur hijau	2024	1	1	140173.1959798995	557
anggur merah gold	2024	1	1	131484.55631399318	395
api anggur hijau	2024	1	1	164322.36224489796	292
*/1. What is the total amount of sales for each product in a specific year?
2. Can we analyze the top-selling product by quantity sold over different months and weeks?
3. Is there a correlation between the total amount and quantity of products sold for each product in a given period?
<EOF>

> Finished chain.
1. What is the total amount of sales for each product in a specifi

In [32]:
tpqs_questions = response['output'].split('<EOF>')[0].split('\n')[:3]
tpqs_questions

['1. What is the total amount of sales for each product in a specific year?',
 '2. Can we analyze the top-selling product by quantity sold over different months and weeks?',
 '3. Is there a correlation between the total amount and quantity of products sold for each product in a given period?']

In [33]:
# create location answer model
tpqs_query = create_sql_agent(analysis_llm, db=db, agent_type="tool-calling")

In [34]:
tpqs_responses = []
for question in tpqs_questions:
    print(question)
    response = tpqs_query.invoke(question + "All prices are in IDR")
    tpqs_responses.append(response['output'])

1. What is the total amount of sales for each product in a specific year?
2. Can we analyze the top-selling product by quantity sold over different months and weeks?
3. Is there a correlation between the total amount and quantity of products sold for each product in a given period?


In [35]:
print(tpqs_responses[0])

The total amount of sales for each product in the year 2024, in IDR, is as follows:

1. Product: Chivas 12yo, Total Sales: 22,505,771.37
2. Product: Jack Daniel's No.7, Total Sales: 17,077,432.33
3. Product: Glenfiddich 12yo, Total Sales: 24,730,651.99
4. Product: Hendricks Gin, Total Sales: 9,208,020.25
5. Product: Hennessy VSOP, Total Sales: 35,864,390.67
6. Product: Grey Goose, Total Sales: 21,013,684.14
7. Product: Bombay Sapphire, Total Sales: 12,256,864.08
8. Product: Monkey Shoulder, Total Sales: 16,176,142.15
9. Product: Johnnie Walker Black Label, Total Sales: 22,612,401.42
10. Product: Martell VSOP, Total Sales: 30,219,499.59

These are the top 10 products with the highest total sales in the year 2024.


In [36]:
print(tpqs_responses[1])

The top-selling product by quantity sold over different months and weeks are as follows:

1. Product: kawa kawa anggur hijau, Year: 2024, Month: 3, Week: 10, Quantity: 624
2. Product: kawa kawa anggur hijau, Year: 2024, Month: 1, Week: 1, Quantity: 557
3. Product: kawa kawa anggur hijau, Year: 2024, Month: 2, Week: 8, Quantity: 534
4. Product: kawa kawa anggur hijau, Year: 2024, Month: 2, Week: 6, Quantity: 520
5. Product: kawa kawa anggur hijau, Year: 2024, Month: 4, Week: 17, Quantity: 455
6. Product: (es batu) crystalline cup, Year: 2024, Month: 2, Week: 6, Quantity: 437
7. Product: kawa kawa anggur hijau, Year: 2024, Month: 2, Week: 5, Quantity: 435
8. Product: kawa kawa anggur hijau, Year: 2024, Month: 3, Week: 9, Quantity: 415
9. Product: kawa kawa anggur hijau, Year: 2024, Month: 3, Week: 13, Quantity: 407
10. Product: draft beer kaleng, Year: 2024, Month: 2, Week: 6, Quantity: 399

These are the top-selling products based on quantity sold over different months and weeks.


In [37]:
print(tpqs_responses[2])

Based on the data from the table "top_product_by_quantity_sold", there seems to be a correlation between the total amount and quantity of products sold for each product in a given period. Here are some examples:

1. Product: "kawa kawa anggur hijau"
   - Total Amount: 140173.20 IDR
   - Quantity: 557

2. Product: "anggur merah gold"
   - Total Amount: 131484.56 IDR
   - Quantity: 395

3. Product: "api anggur hijau"
   - Total Amount: 164322.36 IDR
   - Quantity: 292

The correlation between the total amount and quantity sold for each product can be further analyzed by examining more data points from the table.


In [38]:
# Function to escape newlines
def escape_newlines(text):
    return text.replace('\n', '\\n')

with open('outputs/tpqs.csv', 'w', newline='', encoding='utf-8') as fopen:
    writer = csv.writer(fopen, quoting=csv.QUOTE_MINIMAL)
    
    # Write header if the file is empty
    fopen.seek(0, os.SEEK_END)
    if fopen.tell() == 0:
        writer.writerow(['id', 'prompt', 'response'])

    # Write each question and response to the CSV file
    for idx in range(len(tpqs_responses)):
        writer.writerow(
            [
                idx + 1, 
                escape_newlines(tpqs_questions[idx]), 
                escape_newlines(tpqs_responses[idx])
            ]
        )

## top product by category sold

In [39]:
# Top Product by Category Sold
# Get average price of category too 

tpcs = iseller_data.groupby(['product_type',  'year', 'month', 'week']).agg({'total_amount': 'mean', 'quantity': 'sum'}).reset_index()\
        .sort_values(by='quantity', ascending=False).sort_values(by=["year", "month", "week"])

In [40]:
conn = sqlalchemy.create_engine('sqlite:////Users/davidrinaldi/Documents/Projects/myk-dashboard/dev/data/temp4.db')
tpqs.to_sql('top_product_by_category_sold', conn, if_exists='replace', index=False)
db = SQLDatabase.from_uri("sqlite:///data/temp4.db")

In [41]:
tpcs_agent = create_sql_agent(query_llm, db=db, agent_type="openai-tools", verbose=True)

In [42]:
response = tpcs_agent.invoke(
    """
    Generate only three additional prompts that could be asked to understand or analysis the data.
        Output the three prompts in list format counting up from 1 (1. prompt\n 2. prompt\n 3. prompt\n). 
        Generate only the prompts in the list, exclude their index.
        End the response with <EOF>.
        Do not respond with an empty result or say "I don't know".
    """
)
print(response['output'])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


top_product_by_category_sold
Invoking: `sql_db_schema` with `{'table_names': 'top_product_by_category_sold'}`



CREATE TABLE top_product_by_category_sold (
	clean_product_name TEXT, 
	year INTEGER, 
	month INTEGER, 
	week BIGINT, 
	total_amount FLOAT, 
	quantity BIGINT
)

/*
3 rows from top_product_by_category_sold table:
clean_product_name	year	month	week	total_amount	quantity
kawa kawa anggur hijau	2024	1	1	140173.1959798995	557
anggur merah gold	2024	1	1	131484.55631399318	395
api anggur hijau	2024	1	1	164322.36224489796	292
*/1. What are the top products sold by category in the year 2024?

2. How does the total amount spent compare for the top products in different months of the year?

3. Is there a noticeable trend in the quantity of the top products sold across different weeks in 2024?

<EOF>

> Finished chain.
1. What are the top products sold by category in the year 2024?

2. How does the 

In [43]:
tpcs_questions = response['output'].split('<EOF>')[0].split('\n')[:3]
tpcs_questions

['1. What are the top products sold by category in the year 2024?',
 '',
 '2. How does the total amount spent compare for the top products in different months of the year?']

In [44]:
# create location answer model
tpcs_query = create_sql_agent(analysis_llm, db=db, agent_type="tool-calling")

In [45]:
tpcs_responses = []
for question in tpcs_questions:
    print(question)
    response = tpcs_query.invoke(question + "All prices are in IDR")
    tpcs_responses.append(response['output'])

1. What are the top products sold by category in the year 2024?

2. How does the total amount spent compare for the top products in different months of the year?


In [46]:
print(tpcs_responses[0])

The top products sold by category in the year 2024 are as follows (in IDR):

1. Product: Hibiki Japanese Harmony, Total Amount: 5,377,000, Quantity: 1
2. Product: Hibiki Japanese Harmony, Total Amount: 4,887,000, Quantity: 2
3. Product: Nikka Whisky from the Barrel Japanese Whisky, Total Amount: 4,545,016, Quantity: 1
4. Product: Ballantines Blended Scotch Whisky, Total Amount: 4,545,016, Quantity: 1
5. Product: Jack Daniel's No.7, Total Amount: 4,259,000, Quantity: 1
6. Product: Hennessy VSOP, Total Amount: 4,196,000, Quantity: 4
7. Product: Monkey Shoulder, Total Amount: 3,947,000, Quantity: 2
8. Product: Sababay Mascetti, Total Amount: 3,944,000, Quantity: 4
9. Product: Roku Gin, Total Amount: 3,937,000, Quantity: 1
10. Product: Martell VSOP, Total Amount: 3,832,488, Quantity: 6


In [47]:
print(tpcs_responses[1])

The database contains a table named "top_product_by_category_sold" with the following columns:
- clean_product_name (TEXT)
- year (INTEGER)
- month (INTEGER)
- week (BIGINT)
- total_amount (FLOAT)
- quantity (BIGINT)

Here are 3 sample rows from the table:
1. Product: kawa kawa anggur hijau, Year: 2024, Month: 1, Week: 1, Total Amount: 140173.20, Quantity: 557
2. Product: anggur merah gold, Year: 2024, Month: 1, Week: 1, Total Amount: 131484.56, Quantity: 395
3. Product: api anggur hijau, Year: 2024, Month: 1, Week: 1, Total Amount: 164322.36, Quantity: 292

Now, I can proceed with querying the database based on the information provided. What specific information or question would you like to know from the "top_product_by_category_sold" table?


In [48]:
print(tpcs_responses[2])

The total amount spent on the top products in different months of the year are as follows (in IDR):

1. In January:
   - Product: Hibiki Japanese Harmony
   - Total Amount Spent: 12,851,000

2. In January:
   - Product: Hennessy VSOP
   - Total Amount Spent: 10,279,674.67

3. In March:
   - Product: Martell VSOP
   - Total Amount Spent: 9,995,843.44

4. In February:
   - Product: Hennessy VSOP
   - Total Amount Spent: 9,652,379.33

5. In April:
   - Product: Martell VSOP
   - Total Amount Spent: 8,816,063.55

6. In April:
   - Product: Hennessy VSOP
   - Total Amount Spent: 8,027,970

7. In January:
   - Product: Nikka Whisky from the Barrel Japanese Whisky
   - Total Amount Spent: 6,996,106

8. In February:
   - Product: Roku Gin
   - Total Amount Spent: 6,949,050

9. In April:
   - Product: Chivas 12yo
   - Total Amount Spent: 6,816,145

10. In March:
    - Product: Hennessy VSOP
    - Total Amount Spent: 6,717,866.67

These are the top products and the total amount spent on them in 

In [49]:
# Function to escape newlines
def escape_newlines(text):
    return text.replace('\n', '\\n')

with open('outputs/tpcs.csv', 'w', newline='', encoding='utf-8') as fopen:
    writer = csv.writer(fopen, quoting=csv.QUOTE_MINIMAL)
    
    # Write header if the file is empty
    fopen.seek(0, os.SEEK_END)
    if fopen.tell() == 0:
        writer.writerow(['id', 'prompt', 'response'])

    # Write each question and response to the CSV file
    for idx in range(len(tpcs_responses)):
        writer.writerow(
            [
                idx + 1, 
                escape_newlines(tpcs_questions[idx]), 
                escape_newlines(tpcs_responses[idx])
            ]
        )

## total quantity sold each month

In [50]:
# Total Quantity sold each month
tqm = iseller_data.groupby(["year", "month", "clean_product_name"])['quantity'].sum().reset_index().sort_values(by=["year", "month"])


In [51]:
conn = sqlalchemy.create_engine('sqlite:////Users/davidrinaldi/Documents/Projects/myk-dashboard/dev/data/temp5.db')
tpqs.to_sql('total_quantity_sold_monthly', conn, if_exists='replace', index=False)
db = SQLDatabase.from_uri("sqlite:///data/temp5.db")

In [52]:
tpm_agent = create_sql_agent(query_llm, db=db, agent_type="openai-tools", verbose=True)

In [53]:
response = tpm_agent.invoke(
    """
    Generate only three additional prompts that could be asked to understand or analysis the data.
        Output the three prompts in list format counting up from 1 (1. prompt\n 2. prompt\n 3. prompt\n). 
        Generate only the prompts in the list, exclude their index.
        End the response with <EOF>.
        Do not respond with an empty result or say "I don't know".
    """
)
print(response['output'])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


total_quantity_sold_monthly
Invoking: `sql_db_schema` with `{'table_names': 'total_quantity_sold_monthly'}`



CREATE TABLE total_quantity_sold_monthly (
	clean_product_name TEXT, 
	year INTEGER, 
	month INTEGER, 
	week BIGINT, 
	total_amount FLOAT, 
	quantity BIGINT
)

/*
3 rows from total_quantity_sold_monthly table:
clean_product_name	year	month	week	total_amount	quantity
kawa kawa anggur hijau	2024	1	1	140173.1959798995	557
anggur merah gold	2024	1	1	131484.55631399318	395
api anggur hijau	2024	1	1	164322.36224489796	292
*/1. What are the top 5 clean product names with the highest total amount sold?
2. Can you provide a breakdown of the total quantity sold monthly for each year?
3. Is there a correlation between the total amount and the quantity sold for each clean product name?
<EOF>

> Finished chain.
1. What are the top 5 clean product names with the highest total amount sold?
2. Can you pro

In [54]:
tpm_questions = response['output'].split('<EOF>')[0].split('\n')[:3]
tpm_questions

['1. What are the top 5 clean product names with the highest total amount sold?',
 '2. Can you provide a breakdown of the total quantity sold monthly for each year?',
 '3. Is there a correlation between the total amount and the quantity sold for each clean product name?']

In [55]:
# create location answer model
tpm_query = create_sql_agent(analysis_llm, db=db, agent_type="tool-calling")

In [56]:
import random

tpm_responses = []
questions = ['What is the correlation between external events (such as holidays, festivals, or economic changes) and monthly sales patterns? (Move to forecasting)', 'Are there any emerging trends or shifts in consumer preferences over the months and years?', 'How does the performance of different products compare across the same time periods (e.g., year-over-year comparisons for the same month)?']
for question in tpm_questions:
    try:
        print(question)
        response = tpm_query.invoke(question + "All prices are in IDR")
        tpm_responses.append(response['output'])
    except:
        question = random.choice(questions)
        print(question)
        response = tpm_query.invoke(question + "All prices are in IDR")
        tpm_responses.append(response['output'])
        

1. What are the top 5 clean product names with the highest total amount sold?
2. Can you provide a breakdown of the total quantity sold monthly for each year?
3. Is there a correlation between the total amount and the quantity sold for each clean product name?
Are there any emerging trends or shifts in consumer preferences over the months and years?


In [57]:
print(tpm_responses[0])

The top 5 clean product names with the highest total amount sold in IDR are:
1. Hennessy VSOP - 35,864,390.67
2. Martell VSOP - 30,219,499.59
3. Hibiki Japanese Harmony - 24,804,258.00
4. Glenfiddich 12YO - 24,730,651.99
5. JW Black Label - 22,612,401.42


In [58]:
print(tpm_responses[1])

The breakdown of the total quantity sold monthly for each year is as follows:
- For the year 2024:
  - Month 1: 26,877 units
  - Month 2: 25,491 units
  - Month 3: 20,171 units
  - Month 4: 21,317 units
  - Month 5: 6,372 units

If you need more information or details, feel free to ask!


In [59]:
print(tpm_responses[2])    

Here are some sample data showing the total amount and quantity sold for different products in January 2024:

1. Product: Kawa Kawa Anggur Hijau, Year: 2024, Month: 1, Total Amount: 140173.20, Quantity: 557
2. Product: Anggur Merah Gold, Year: 2024, Month: 1, Total Amount: 131484.56, Quantity: 395
3. Product: Api Anggur Hijau, Year: 2024, Month: 1, Total Amount: 164322.36, Quantity: 292
4. Product: Anggur Merah, Year: 2024, Month: 1, Total Amount: 118656.69, Quantity: 231
5. Product: Bintang Kaleng, Year: 2024, Month: 1, Total Amount: 222735.70, Quantity: 224
6. Product: (Es Batu) Crystalline Cup, Year: 2024, Month: 1, Total Amount: 157670.77, Quantity: 205
7. Product: Draft Beer Kaleng, Year: 2024, Month: 1, Total Amount: 155940.31, Quantity: 202
8. Product: Anggur Ginseng Intisari, Year: 2024, Month: 1, Total Amount: 119212.61, Quantity: 192
9. Product: Bintang Botol, Year: 2024, Month: 1, Total Amount: 222384.00, Quantity: 181
10. Product: Iceland Vodka, Year: 2024, Month: 1, Total 

In [60]:
# Function to escape newlines
def escape_newlines(text):
    return text.replace('\n', '\\n')

with open('outputs/tpm.csv', 'w', newline='', encoding='utf-8') as fopen:
    writer = csv.writer(fopen, quoting=csv.QUOTE_MINIMAL)
    
    # Write header if the file is empty
    fopen.seek(0, os.SEEK_END)
    if fopen.tell() == 0:
        writer.writerow(['id', 'prompt', 'response'])

    # Write each question and response to the CSV file
    for idx in range(len(tpm_responses)):
        writer.writerow(
            [
                idx + 1, 
                escape_newlines(tpm_questions[idx]), 
                escape_newlines(tpm_responses[idx])
            ]
        )

## total quantity sold each day

In [62]:
# Total Quantity sold each Day
tqd = iseller_data.groupby(['year', 'month', 'week', 'day_name', 'day_of_week_num', 'clean_product_name'])['quantity'].sum().reset_index().sort_values(by="day_of_week_num")


KeyError: 'day_name'

# advanced query response examples

In [ ]:
print(tpm_query.invoke('Are there any emerging trends or shifts in consumer preferences over the months and years?')['output'])

Based on the data from the `total_quantity_sold_monthly` table, here are some emerging trends or shifts in consumer preferences over the months and years:

1. In May 2024, the top-selling products were "kawa kawa anggur hijau" with a total amount of 170459.99 and 308 quantity sold.
2. Following that, "anggur merah gold" was also popular in May 2024 with a total amount of 190785.17 and 231 quantity sold.
3. "bintang kaleng" saw an increase in sales in May 2024 with a total amount of 219800.71 and 179 quantity sold.

These insights indicate changes in consumer preferences over the months and years, with certain products gaining popularity.


In [ ]:
test = query_llm.invoke("""
Are there any emerging trends or shifts in consumer preferences over the months and years?

Based on the data from the `total_quantity_sold_monthly` table, here are some emerging trends or shifts in consumer preferences over the months and years:

1. In May 2024, the top-selling products were "kawa kawa anggur hijau" with a total amount of 170459.99 and 308 quantity sold.
2. Following that, "anggur merah gold" was also popular in May 2024 with a total amount of 190785.17 and 231 quantity sold.
3. "bintang kaleng" saw an increase in sales in May 2024 with a total amount of 219800.71 and 179 quantity sold.

These insights indicate changes in consumer preferences over the months and years, with certain products gaining popularity.
""")

In [ ]:
print(test.content)


4. The data also shows a decrease in sales for some products, such as "teh botol" and "es kopi susu" in May 2024 compared to previous months. This could indicate a shift in consumer preferences towards other beverage options.
5. There is a trend of increased sales for healthier or more premium products, such as "anggur merah gold", which could suggest a growing interest in quality or higher-end products among consumers.
6. Seasonal preferences may also play a role, as seen in the spike in sales of "kawa kawa anggur hijau" in May 2024, which could be influenced by factors like warmer weather or special occasions during that month.

Overall, the data highlights the dynamic nature of consumer preferences and the importance of staying attuned to market trends to meet changing demands.


In [ ]:
test = sales_query.invoke('''Identifying emerging products with high growth potential allows the business to capitalize on new opportunities and allocate resources effectively to support their growth.''')

In [ ]:
print(test['output'])

The query results show products with high growth potential based on the `quarterly_growth` and `monthly_growth` percentages:

1. Product: bisquit & dubouche vsop, Product Type: COGNAC, Quarterly Growth: Infinity, Monthly Growth: Infinity
2. Product: paket 1 anggur merah gold  + 1 sampoerna mild merah 16, Product Type: WINE, Quarterly Growth: Infinity, Monthly Growth: Infinity
3. Product: paket 1 api anggur hijau  + 1 sampoerna mild merah 16, Product Type: WINE, Quarterly Growth: Infinity, Monthly Growth: Infinity
4. Product: singaraja apidin kaleng - resell dus, Product Type: BEER, Quarterly Growth: Infinity, Monthly Growth: Infinity
5. Product: paket 2 + free 1 happy soju, Product Type: SOJU, Quarterly Growth: Infinity, Monthly Growth: 470.40%
6. Product: paket 1 kawa kawa anggur hijau  + 1 sampoerna mild merah 16, Product Type: WINE, Quarterly Growth: Infinity, Monthly Growth: 336.82%
7. Product: hatten sweet alexandria, Product Type: WINE, Quarterly Growth: Infinity, Monthly Growth: